In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import json

import IPython
import requests
import dotenv
import pandas as pd
import numpy as np

from backend.location import Location, spherical_distance
from backend import gmaps
from backend.grid import make_grid, Grid

dotenv.load_dotenv()

In [ ]:
GMAPS_API_KEY = os.environ["GMAPS_API_KEY"]

In [ ]:
# zurich = Location(47.378, 8.540)
# zurich = Location(47.369, 8.5415)
zurich = Location(47.375, 8.5415)
zoom = 14
size_pixels = 640
# grid = make_grid(zurich, zoom, size=5)

grid = Grid(zurich, zoom=zoom, size=20, snap_to_roads=True, size_pixels=size_pixels)

image = gmaps.get_static_map(zurich, zoom, markers=grid.get_snapped_locations(), size_pixels=size_pixels)
# image = gmaps.get_static_map(zurich, zoom, markers=[], size_pixels=size_pixels)
IPython.display.Image(image)

In [ ]:
image = gmaps.get_static_map(zurich, zoom, markers=grid.get_snapped_locations(), size_pixels=size_pixels)
IPython.display.Image(image)

In [ ]:
image = gmaps.get_static_map(zurich, zoom, markers=[])
IPython.display.Image(image)

In [ ]:
matrix = list(gmaps.get_distance_matrix(grid.get_snapped_locations(), grid.get_snapped_locations()))

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
grid.compute_sparsified_distance_matrix(max_normalized_distance=0.12)

In [ ]:
with open("20x20grid-v8.json", "w") as f:
    json.dump(grid.to_json(), f)

In [ ]:
with open("15x15grid-v7.json", "w") as f:
    json.dump(grid.to_json(), f)

In [ ]:
with open("5x5matrix-v3-request.json", "w") as f:
    json.dump(gmaps.get_distance_matrix_api_payload(grid, grid), f)

In [ ]:
[x for x in matrix if "destinationIndex" not in x]

## Analysis

In [ ]:
with open("../frontend/src/assets/9x9matrix-v2.json") as f:
    matrix = json.load(f)

In [ ]:
df = pd.DataFrame(matrix)
df = df.set_index(["originIndex", "destinationIndex"]).sort_index()

In [ ]:
df["durationSec"] = df["duration"].apply(lambda x: x if pd.isna(x) else int(x[:-1]))

In [ ]:
df["originLat"] = df.index.get_level_values(0).map(lambda i: grid[i].lat)
df["originLng"] = df.index.get_level_values(0).map(lambda i: grid[i].lng)
df["destinationLat"] = df.index.get_level_values(1).map(lambda i: grid[i].lat)
df["destinationLng"] = df.index.get_level_values(1).map(lambda i: grid[i].lng)

In [ ]:
df

In [ ]:
df["sphericalDistance"] = df.apply(
    lambda row: spherical_distance(
        Location(row.originLat, row.originLng),
        Location(row.destinationLat, row.destinationLng),
    ),
    axis=1,
)

In [ ]:
df["distanceRatio"] =  df["sphericalDistance"] / df["distanceMeters"]
df["timeRatio"] =  df["sphericalDistance"] / df["durationSec"]

In [ ]:
df.sort_values("distanceRatio", ascending=False)

In [ ]:
import plotly.express as px

In [ ]:
reindexing
